In [ ]:
%pip install --upgrade semantic-kernel

# Add your environment variables before running the following cell

In [2]:
%%writefile .env
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=""
AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME=""
AZURE_OPENAI_DEPLOYMENT_NAME=""
AZURE_OPENAI_ENDPOINT=""
AZURE_OPENAI_API_KEY=""
AZCOSMOS_API="mongo-vcore" # others are not supported yet
AZCOSMOS_CONNSTR=""
AZCOSMOS_DATABASE_NAME=""
AZCOSMOS_CONTAINER_NAME=""

Writing .env-test


In [4]:
# load the environment variables file
from dotenv import dotenv_values

config = dotenv_values(".env")

In [5]:
database_name = config.get("AZCOSMOS_DATABASE_NAME")
collection_name = config.get("AZCOSMOS_CONTAINER_NAME")

In [6]:
index_name = "VectorSearchIndex"
vector_dimensions = 1536 # text-embedding-ada-002 uses a 1536-dimensional embedding vector
num_lists = 1
similarity = "COS"

In [7]:
import json

async def upsert_data_to_memory_store(memory_store: callable, data_file_path: str):
    with open(file=data_file_path, mode="r", encoding="utf-8") as f:
        data = json.load(f)
        n = 0
        for item in data:
            n+=1
            await memory_store.save_information(
                collection=collection_name,
                id=item["id"],
                text=item["content"],
                description=item["title"]
            )
            print("Generating embeddings and saving item:", n, "/" ,len(data), end='\r')

In [10]:
import semantic_kernel as sk

# get api key and endpoint from .env file
_, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel = sk.Kernel()

In [11]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)

Registering Azure OpenAI Chat Service...


In [ ]:
# adding azure openai chat service
chat_model_deployment_name = config.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

azure_chat_service = AzureChatCompletion(
    deployment_name=chat_model_deployment_name,
    endpoint=endpoint,
    api_key=api_key
)

In [12]:
kernel.add_chat_service(chat_model_deployment_name, azure_chat_service)
print("Added Azure OpenAI Chat Service...")

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat-model': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x000001F1FFB86E60>}, chat_services={'chat-model': <function Kernel.add_chat_service.<locals>.<lambda> at 0x000001F1FFB86DD0>}, text_embedding_generation_services={}, default_text_completion_service='chat-model', default_chat_service='chat-model', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [13]:
# adding azure openai text embedding service
embedding_model_deployment_name = config.get("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME")

azure_text_embedding_service = AzureTextEmbedding(
    deployment_name=embedding_model_deployment_name,
    endpoint=endpoint,
    api_key=api_key
)

Registering Azure OpenAI Embedding Service...


In [14]:
kernel.add_text_embedding_generation_service(embedding_model_deployment_name, azure_text_embedding_service)
print("Added Azure OpenAI Embedding Generation Service...")

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat-model': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x000001F1FFB86E60>}, chat_services={'chat-model': <function Kernel.add_chat_service.<locals>.<lambda> at 0x000001F1FFB86DD0>}, text_embedding_generation_services={'embedding-model': <function Kernel.add_text_embedding_generation_service.<locals>.<lambda> at 0x000001F1F91C5BD0>}, default_text_completion_service='chat-model', default_chat_service='chat-model', default_text_embedding_generation_service='embedding-model', retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [22]:
from semantic_kernel.connectors.memory.azure_cosmosdb import (
    AzureCosmosDBMemoryStore,
)

print("Creating or updating Azure Cosmos DB Memory Store...")
# create azure cosmos db for mongo db vcore api store and collection with vector ivf
# currently SK only supports the ivf vector kind
store  = await AzureCosmosDBMemoryStore.create(
    database_name=database_name,
    collection_name=collection_name,
    index_name=index_name,
    vector_dimensions=vector_dimensions,
    num_lists=num_lists,
    similarity=similarity
)
print("Finished updating Azure Cosmos DB Memory Store...")

Creating or updating Azure Cosmos DB Memory Store...
Finished updating Azure Cosmos DB Memory Store...


In [23]:
kernel.register_memory_store(memory_store=store)
print("Registered Azure Cosmos DB Memory Store...")

Registered Azure Cosmos DB Memory Store...


In [24]:
print("Upserting data to Azure Cosmos DB Memory Store...")
await upsert_data_to_memory_store(kernel.memory, "./text-sample.json")

Upserting data to Azure Cosmos DB Memory Store...


In [147]:
# each time it calls the embedding model to generate embeddings from your query
query_term = "What is Azure Database for Managed Instances?"
result = await kernel.memory.search(collection_name, query_term)

In [148]:
print(f"Result is: {result[0].text}\nRelevance Score: {result[0].relevance}\nFull Record: {result[0].additional_metadata}")

Result is: Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. SQL Managed Instance supports various data types, such as JSON, spatial, and full-text. You can use Azure SQL Managed Instance to migrate your existing applications, build new applications, and ensure the performance and security of your data. It also integrates with other Azure services, such as Azure App Service and Azure Data Factory.
Relevance Score: 0.8967287909762478
Full Record: {"text": "Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. SQL Managed Instance supports various data types, such as JSON, spatial, and full-text. You can use Azure SQL Managed Instance to migrate your existing applications, build new applications, and ensure the performance and

In [152]:
prompt = """
    You are a chatbot that can have a conversations about any topic related to the provided context.
    Start by saying how relevant the question is using the provided context relevancy score.
    Give explicit answers from the provided context or say 'I don't know' if it does not have an answer.
    provided context: {{$db_record}}

    User: {{$query_term}}
    Chatbot:"""

In [153]:
chat_function = kernel.create_semantic_function(prompt, max_tokens=500, temperature=0.0, top_p=0.5)
context = kernel.create_new_context()

In [154]:
context['query_term'] = query_term
context['db_record'] = result[0].additional_metadata
completions_result = await chat_function.invoke(context=context)

In [155]:
print(completions_result)

The provided context is highly relevant to your question. Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hosted in Azure. It provides features like automatic backups, monitoring, and high availability. It is a database service that allows you to migrate your existing applications, build new applications, and ensure the performance and security of your data.


In [168]:
import time
query_term = ""
while True and query_term != "exit":
    query_term = input("Enter a query: ")
    result = await kernel.memory.search(collection_name, query_term)
    context['query_term'] = query_term
    context['db_record'] = result[0].additional_metadata
    completions_result = await chat_function.invoke(context=context)
    print(f"Question:\n{query_term}\nResponse:\n{completions_result}")
    time.sleep(10)

Question: Hey how are you?
Response: The provided context is about Azure Event Hubs. The question is not relevant to the context. Can I help you with anything related to Azure Event Hubs?
Question: What is Azure App Service?
Response: The question is highly relevant with a score of 1. Azure App Service is a fully managed platform for building, deploying, and scaling web apps. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. You can host web apps, mobile app backends, and RESTful APIs on it. It also offers built-in auto-scaling and load balancing capabilities and integrates with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.
Question: exit
Response: Goodbye!
